In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id


In [3]:
logfile = 'name_list'## name your log file.
connector = Connector(logfile)

In [4]:
url = 'https://www.imdb.com/list/ls022928836/'
response,call_id = connector.get(url,'finding_actress_names')
if response.ok:
    html = response.text
else:
    print('error')


In [5]:
#links = set()

#for link_loc in html.split('href="')[1:]:
#    link = link_loc.split('"')[0]
#    if '/names/' in link:
#        links.add(link)


#html.split('href="')[3:]

# Actor and actress names

In [60]:
soup= BeautifulSoup(html,'html.parser')

first_male=[]
sur_male=[]
full_m=[]

for i in range(1,8): 
    url = 'https://www.imdb.com/list/ls022928819/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call = 'Exam, KU{}'.format(i)
    response,call_id = connector.get(url, call)
    if response.ok:
        html = response.text
    else:
        print('error')
    soup= BeautifulSoup(html,'html.parser')
    if i < 7:
        for j in range(0,100): 
            first_male.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[0])
            sur_male.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[1:])
            full_m.append(soup.findAll('h3')[j].text.split('\n')[2].strip())
    else :
        for j in range(0,81): 
            first_male.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[0])
            sur_male.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[1:])
            full_m.append(soup.findAll('h3')[j].text.split('\n')[2].strip())


['Josef', 'Liefers']

In [49]:
url = 'https://www.imdb.com/list/ls022928819/?sort=list_order,asc&mode=detail&page=1'
call = 'Exam, KU{}'.format(i)
response,call_id = connector.get(url, call)
if response.ok:
    html = response.text
else:
    print('error')

soup.findAll('h3')[1].text.split('\n')[2].strip().split(' ')[1]

'Helberg'

In [61]:
first_female=[]
sur_female=[]
full_f=[]

for i in range(1,8): 
    url = 'https://www.imdb.com/list/ls022928836/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call = 'Exam, KU{}'.format(i)
    response,call_id = connector.get(url, call)
    if response.ok:
        html = response.text
    else:
        print('error')
    soup= BeautifulSoup(html,'html.parser')
    
    if i < 7:
        for j in range(0,100): 
            first_female.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[0])
            sur_female.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[1:])
            full_f.append(soup.findAll('h3')[j].text.split('\n')[2].strip())
    else :
        for j in range(0,81): 
            first_female.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[0])
            sur_female.append(soup.findAll('h3')[j].text.split('\n')[2].strip().split(' ')[1:])
            full_f.append(soup.findAll('h3')[j].text.split('\n')[2].strip())


IndexError: list index out of range

# Directors names 

## Female

In [62]:
soup= BeautifulSoup(html,'html.parser')

female_name_director = []

url = 'https://www.imdb.com/list/ls003532091/?fbclid=IwAR36QhJh8EDYKPCh_9RTsJdkvjNf-c-xQxtGy8bUZz1E2HefYpzxWeFA5w0'
call = 'Exam, KU{}'.format(i)
response,call_id = connector.get(url, call)
if response.ok:
    html = response.text
else:
    print('error')
soup= BeautifulSoup(html,'html.parser')
    
for j in range(0,100): 
    tree_node = soup.findAll('h3')[j]
    name = tree_node.text.split('\n')[2].strip()
    female_name_director.append(name)


In [63]:
female_name_director[0]

'Vera Chytilová'

## Male

In [64]:
soup= BeautifulSoup(html,'html.parser')

male_name_director = []
url = 'https://www.imdb.com/list/ls058727091/'
call = 'Exam, KU{}'.format(i)
response,call_id = connector.get(url, call)
if response.ok:
    html = response.text
else:
    print('error')
soup= BeautifulSoup(html,'html.parser')
    
for j in range(0,100): 
    tree_node = soup.findAll('h3')[j]
    name = tree_node.text.split('\n')[2].strip()
    male_name_director.append(name)

In [17]:
male_name_director[0]

'Stanley Kubrick'

In [19]:
#numpy.savetxt('Output.txt', ["Purchase Amount: %s" % TotalAmount], fmt='%s')

print(female_name,  file=open('female_name.txt', 'w'))
print(male_name, file = open('male_name.txt','w'))
print(male_name_director, file=open('male_name_director.txt', 'w'))
print(female_name_director, file=open('female_name_director.txt', 'w'))

# Finding genres

In [17]:
movie_genre= []
movie_summary = []
movie_stars = []

for v in range(1,201):
    url1 = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page={}&title_type=movie&fbclid=IwAR3B7G9VdhKWjVvbFIPhdH9vGLZwmO_zzTeNlCj4whUMbn_RtS3g1g9FiUQ&release_date=1980%2C2019&sort=num_votes,desc'.format(v)
    call = 'Exam, KU{}'.format(v)
    response,callid = connector.get(url1, call)
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    
    for j in range(50):
        if soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("span", {"class":"genre"}) ==[]:
            movie_genre.append(None)
        else: 
            genre = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("span", {"class":"genre"})[0].text.strip()
            movie_genre.append(genre)
        if soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})== []:
            movie_summary.append(None)
        else: 
            summary = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})[0].text.strip()
            movie_summary.append(summary)
        if soup.findAll('div',{"class":'lister-item mode-detail'})[0].findAll("p", {"class":"text-muted text-small"})== []:
            movie_stars.append(None)
        else: 
            stars = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":"text-muted text-small"})[1].text.strip()
            stars_1 = stars.split('Stars')[1].split('\n')[1:5]
            movie_stars.append(stars_1)

In [19]:
print(len(movie_genre))
print(len(movie_summary))
print(len(movie_stars))

50
50
50


# Finding Summary

In [253]:
movie_summary = []

for v in range(1, 3):
    url1 = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_typ&sort=moviemeter%2Casc&mode=detail&page={}&title_type=movie&fbclid=IwAR3AAeVD_whYi1AwPsSGdYsgH8FnupVrBhImRFD__GL3wdHegJwcxrUsfyc'.format(v)
    call = 'Exam, KU{}'.format(v)
    response,callid = connector.get(url1, call)
    html = response.text
    soup = BeautifulSoup(html,'lxml')

    for j in range(0, 50): 
        if soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})== []:
            movie_summary.append(None)
        else: 
            summary = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})[0].text.strip()
            movie_summary.append(summary)



In [255]:
print(len(movie_summary))
movie_summary[0]

100


"A faded television actor and his stunt double strive to achieve fame and success in the film industry during the final years of Hollywood's Golden Age in 1969 Los Angeles."

# Finding director

In [37]:
movie_director = []

for v in range(1, 3):
    url1 = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_typ&sort=moviemeter%2Casc&mode=detail&page={}&title_type=movie&fbclid=IwAR3AAeVD_whYi1AwPsSGdYsgH8FnupVrBhImRFD__GL3wdHegJwcxrUsfyc'.format(v)
    call = 'Exam, KU{}'.format(v)
    response,callid = connector.get(url1, call)
    html = response.text
    soup = BeautifulSoup(html,'lxml')

    for j in range(0, 50): 
        if soup.findAll('div',{"class":'lister-item mode-detail'})[0].findAll("p", {"class":"text-muted text-small"})== []:
            movie_director.append(None)
        else: 
            director = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":"text-muted text-small"})[1].text.strip()
            director_1 = director.split('Stars')[0].split('\n')[1]
            movie_director.append(director_1)
            

In [39]:
len(movie_director)

100

# Finding movie stars

In [49]:
movie_stars = []

for v in range(1, 3):
    url1 = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_typ&sort=moviemeter%2Casc&mode=detail&page={}&title_type=movie&fbclid=IwAR3AAeVD_whYi1AwPsSGdYsgH8FnupVrBhImRFD__GL3wdHegJwcxrUsfyc'.format(v)
    call = 'Exam, KU{}'.format(v)
    response,callid = connector.get(url1, call)
    html = response.text
    soup = BeautifulSoup(html,'lxml')

    for j in range(0, 50): 
        if soup.findAll('div',{"class":'lister-item mode-detail'})[0].findAll("p", {"class":"text-muted text-small"})== []:
            movie_stars.append(None)
        else: 
            stars = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":"text-muted text-small"})[1].text.strip()
            stars_1 = stars.split('Stars')[1].split('\n')[1:5]
            movie_stars.append(stars_1)

In [55]:
len(movie_stars)

100

# Finding everything at once

In [39]:
movie_genre= []
movie_summary = []
movie_stars = []

for v in range(1,201):
    url1 = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page={}&title_type=movie&fbclid=IwAR3B7G9VdhKWjVvbFIPhdH9vGLZwmO_zzTeNlCj4whUMbn_RtS3g1g9FiUQ&release_date=1980%2C2019&sort=num_votes,desc'.format(v)
    call = 'Exam, KU{}'.format(v)
    response,callid = connector.get(url1, call)
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    print(v)
    for j in range(50):
        if soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("span", {"class":"genre"}) ==[]:
            movie_genre.append(None)
        else: 
            genre = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("span", {"class":"genre"})[0].text.strip()
            movie_genre.append(genre)
        if soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})==[]:
            movie_summary.append(None)
        else: 
            summary = soup.findAll('div',{"class":'lister-item mode-detail'})[j].findAll("p", {"class":""})[0].text.strip()
            movie_summary.append(summary)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166


KeyboardInterrupt: 

In [60]:
df_summary = pd.DataFrame(movie_summary)
df_genre = pd.DataFrame(movie_genre)

In [61]:
df_summary['genre']=df_genre
df_summary.columns = ['summary', 'genre']

In [62]:
df_summary

,summary,genre
0,Two imprisoned men bond over a number of years...,Drama
1,When the menace known as The Joker emerges fro...,"Action, Crime, Drama"
2,A thief who steals corporate secrets through t...,"Action, Adventure, Sci-Fi"
3,An insomniac office worker and a devil-may-car...,Drama
4,"The lives of two mob hitmen, a boxer, a gangst...","Crime, Drama"
5,"The presidencies of Kennedy and Johnson, the e...","Drama, Romance"
6,A computer hacker learns from mysterious rebel...,"Action, Sci-Fi"
7,A meek Hobbit from the Shire and eight compani...,"Adventure, Drama, Fantasy"
8,Gandalf and Aragorn lead the World of Men agai...,"Adventure, Drama, Fantasy"
9,Eight years after the Joker's reign of anarchy...,"Action, Thriller"
